In [1]:
import pandas as pd
import json
import gc
from tqdm import tqdm

# Books

In [2]:
data = pd.read_csv('../data/google_api_4.tsv', sep='\t', error_bad_lines=False)

/tmp/ipykernel_109598/1935821853.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('../data/google_api_4.tsv', sep='\t', error_bad_lines=False)
b'Skipping line 8396: expected 10 fields, saw 15\nSkipping line 18821: expected 10 fields, saw 15\nSkipping line 20288: expected 10 fields, saw 15\nSkipping line 29800: expected 10 fields, saw 15\nSkipping line 45351: expected 10 fields, saw 15\nSkipping line 54853: expected 10 fields, saw 15\nSkipping line 55256: expected 10 fields, saw 18\nSkipping line 58152: expected 10 fields, saw 15\n'
b'Skipping line 73245: expected 10 fields, saw 15\nSkipping line 73878: expected 10 fields, saw 15\nSkipping line 83732: expected 10 fields, saw 15\nSkipping line 84935: expected 10 fields, saw 15\nSkipping line 85700: expected 10 fields, saw 18\nSkipping line 90157: expected 10 fields, saw 15\nSkipping line 92526: expected 10 fields, saw 15\n

In [3]:
data

,ISBN,authors,publisher,publish_date,description,page_cnt,categories,average_rating,rating_count,synopsis
0,0001713353,"['Nancy Gurney', 'Eric Gurney']",Harpercollins Pub Limited,1986-03-01,Lower primary In the style of Dr. Seuss.,64,['Animals'],None,None,"THE KING, THE MICE AND THE CHEESE follows a fo..."
1,0001061240,None,None,None,None,None,None,None,None,None
2,0001712799,['Dr. Seuss'],HarperCollins Publishers,1982-04,Synopsis coming soon.......,28,['Bedtime'],5,4,Chinese/English edition of Great Day for Up Dr...
3,0002005263,['Tony Hillerman'],HarperCollins,2003,Tony Hillerman is a man at the top of his game...,228,"['Chee, Jim (Fictitious character)']",3,16,"Hillerman's blockbuster ""New York Times"" bests..."
4,0001384198,"['Keith Moseley', 'Mabel Caroline Bragg']",HarperCollins,1984,None,8,['Engines'],4,78,None
...,...,...,...,...,...,...,...,...,...,...
197363,0692699422,['Evan Tarver'],Evantarver.com,2015-11-22,A modern fiction novel with very nonfiction pr...,318,[],None,None,A modern fiction novel with very nonfiction pr...
197364,1548371270,"['JoJo San Diego', 'Jerry Corley']",Createspace Independent Publishing Platform,2017-06-26,Did you know Jack and Jill have become zombie ...,28,['Nursery rhymes'],None,None,Did you know Jack and Jill have become zombie ...
197365,1929997159,None,None,None,None,None,None,None,None,None
197366,0545935172,None,None,None,None,None,None,None,None,None


In [4]:
data.columns

Index(['ISBN', 'authors', 'publisher', 'publish_date', 'description',
       'page_cnt', 'categories', 'average_rating', 'rating_count', 'synopsis'],
      dtype='object')

In [5]:
for i in tqdm(data.index) :
    if data.loc[i,'categories'] == '[]':
        data.loc[i,'categories'] = "['etc']"
    if (data.loc[i,'synopsis'] == 'None') or (data.loc[i,'authors'] == 'None') :
        data.drop(i, inplace=True)

100%|██████████| 197368/197368 [40:32<00:00, 81.15it/s] 


In [6]:
def load_json_data():
    gc.collect()
    with open(f'../data/meta_books.json', 'r') as json_file :
        for line in json_file :
            data = json.loads(line)
            yield data['asin'], data['title']

In [7]:
def load_url_data():
    gc.collect()
    with open(f'../data/img_url_ver2.json', 'r') as json_file :
        for line in json_file :
            data = json.loads(line)
            yield data['isbn'], data['url']

In [8]:
title_arr = []
i = 0
for line in tqdm(load_json_data()):
    title_arr.append(line)

2934949it [00:50, 57919.50it/s]


In [9]:
title_df = pd.DataFrame(title_arr, columns=['id','title'])

In [10]:
print('total_len :', len(data))
for col in ['publisher', 'publish_date', 'page_cnt']:
    cnt = 0
    for i in data[col]:
        if i == 'None':
            cnt += 1
    print(col + ' : ' + str(cnt))

total_len : 121564
publisher : 33297
publish_date : 103
page_cnt : 3871


In [11]:
title_df

,id,title
0,0000092878,Biology Gods Living Creation Third Edition 10 ...
1,000047715X,Mksap 16 Audio Companion: Medical Knowledge Se...
2,0000004545,"Flex! Discography of North American Punk, Hard..."
3,0000013765,Heavenly Highway Hymns: Shaped-Note Hymnal
4,0000000116,Georgina Goodman Nelson Womens Size 8.5 Purple...
...,...,...
2934944,B01HJBPTUI,Made Men: A Thriller (Law of Retaliation Book ...
2934945,B01HJC63ZM,Raptor&#39;s Desire (A Planet Desire novelette...
2934946,B01HJEB422,"LG K4 Case,LG Optimus Zone 3 Case,LG Spree Cas..."
2934947,B01HJDS76Y,Magickal Incantations


In [12]:
books = data.merge(title_df, left_on='ISBN', right_on='id')

In [13]:
url_arr = []
for line in tqdm(load_url_data()):
    url_arr.append(line)

2603354it [00:08, 297842.69it/s]


In [14]:
books

,ISBN,authors,publisher,publish_date,description,page_cnt,categories,average_rating,rating_count,synopsis,id,title
0,0001713353,"['Nancy Gurney', 'Eric Gurney']",Harpercollins Pub Limited,1986-03-01,Lower primary In the style of Dr. Seuss.,64,['Animals'],None,None,"THE KING, THE MICE AND THE CHEESE follows a fo...",0001713353,"King, the Mice and the Cheese (Beginner Books)"
1,0001712799,['Dr. Seuss'],HarperCollins Publishers,1982-04,Synopsis coming soon.......,28,['Bedtime'],5,4,Chinese/English edition of Great Day for Up Dr...,0001712799,Great Day for Up
2,0002005263,['Tony Hillerman'],HarperCollins,2003,Tony Hillerman is a man at the top of his game...,228,"['Chee, Jim (Fictitious character)']",3,16,"Hillerman's blockbuster ""New York Times"" bests...",0002005263,The Sinister Pig
3,0001381733,"['Robert Louis Stevenson', 'Gyo Fujikawa']",HarperCollins,1972,A classic collection of Stevenson's childrens ...,110,['English poetry'],4,41,Brian Wildsmith's stunning art is the backdrop...,0001381733,A Child's Garden of Verses
4,0001711296,"['Theo LeSieg', 'Dr. Seuss']",Collins,1967,Children visit many different houses throughou...,63,"[""Children's poetry, American""]",None,None,This title belongs to the highly acclaimed Beg...,0001711296,Come Over to My House (I Can Read It All by My...
...,...,...,...,...,...,...,...,...,...,...,...,...
121559,1947279424,['Renee Nickell'],Renee Nickell,2018-06-14,Her world came crashing down when she got the ...,284,['Biography & Autobiography'],None,None,Her world came crashing down when she got the ...,1947279424,Always My Hero: The Road to Hope &amp; Healing...
121560,0615622526,"['Felice Austin', 'Lani Axman', 'Heather Farre...",None,2012-04-01,Pregnancy and childbirth are not to be feared;...,542,['Childbirth'],5,1,Pregnancy and childbirth are not to be feared;...,0615622526,The Gift of Giving Life: Rediscovering the Div...
121561,0692699422,['Evan Tarver'],Evantarver.com,2015-11-22,A modern fiction novel with very nonfiction pr...,318,['etc'],None,None,A modern fiction novel with very nonfiction pr...,0692699422,Life in Twenty-Something: A Story of Self-Disc...
121562,1548371270,"['JoJo San Diego', 'Jerry Corley']",Createspace Independent Publishing Platform,2017-06-26,Did you know Jack and Jill have become zombie ...,28,['Nursery rhymes'],None,None,Did you know Jack and Jill have become zombie ...,1548371270,Mother Ghoul's Zombie Nursery Rhymes: Funny Zo...


In [15]:
url_df = pd.DataFrame(url_arr, columns=['isbn','image_URL'])

In [16]:
books = books.merge(url_df, left_on='ISBN', right_on='isbn')

In [17]:
books['publication_year'] = books['publish_date'].str[:4]

In [18]:
books = books[['id', 'image_URL', 'title', 'publisher', 'publication_year', 'synopsis']]

In [19]:
books

,id,image_URL,title,publisher,publication_year,synopsis
0,0001713353,https://pictures.abebooks.com/isbn/97800017133...,"King, the Mice and the Cheese (Beginner Books)",Harpercollins Pub Limited,1986,"THE KING, THE MICE AND THE CHEESE follows a fo..."
1,0001712799,https://pictures.abebooks.com/isbn/97800017127...,Great Day for Up,HarperCollins Publishers,1982,Chinese/English edition of Great Day for Up Dr...
2,0002005263,https://pictures.abebooks.com/isbn/97800020052...,The Sinister Pig,HarperCollins,2003,"Hillerman's blockbuster ""New York Times"" bests..."
3,0001381733,https://pictures.abebooks.com/isbn/97800013817...,A Child's Garden of Verses,HarperCollins,1972,Brian Wildsmith's stunning art is the backdrop...
4,0001711296,https://pictures.abebooks.com/isbn/97800017112...,Come Over to My House (I Can Read It All by My...,Collins,1967,This title belongs to the highly acclaimed Beg...
...,...,...,...,...,...,...
121559,1947279424,https://pictures.abebooks.com/isbn/97819472794...,Always My Hero: The Road to Hope &amp; Healing...,Renee Nickell,2018,Her world came crashing down when she got the ...
121560,0615622526,https://pictures.abebooks.com/isbn/97806156225...,The Gift of Giving Life: Rediscovering the Div...,None,2012,Pregnancy and childbirth are not to be feared;...
121561,0692699422,https://pictures.abebooks.com/isbn/97806926994...,Life in Twenty-Something: A Story of Self-Disc...,Evantarver.com,2015,A modern fiction novel with very nonfiction pr...
121562,1548371270,https://pictures.abebooks.com/isbn/97815483712...,Mother Ghoul's Zombie Nursery Rhymes: Funny Zo...,Createspace Independent Publishing Platform,2017,Did you know Jack and Jill have become zombie ...


In [20]:
version = 'ver3'

In [22]:
books.to_csv(f'../data/{version}/Book.tsv', sep='\t', index=False)

# Author

In [23]:
import ast

In [24]:
authors = data[['ISBN', 'authors']]
aut_arr = []
for idx, row in authors.iterrows():
    try :
        for a in ast.literal_eval(row['authors']):
            aut_arr.append((row['ISBN'], a))
    except :
        continue

In [25]:
aut_df = pd.DataFrame(aut_arr, columns=['book_id','author_id'])

In [26]:
from sklearn.preprocessing import LabelEncoder

In [27]:
enc = LabelEncoder()

In [28]:
auts = aut_df['author_id']

In [29]:
aut_df['author_id'] = enc.fit_transform(aut_df['author_id'])

In [30]:
aut_df

,book_id,author_id
0,0001713353,51722
1,0001713353,21200
2,0001712799,18956
3,0002005263,69836
4,0001381733,59398
...,...,...
140302,0615622526,59889
140303,0692699422,21824
140304,1548371270,32887
140305,1548371270,31949


In [31]:
aut_df_t = pd.DataFrame()

In [32]:
aut_df_t['id'] = aut_df['author_id']

In [78]:
aut_df_t['name'] = auts

In [79]:
aut_df_t= aut_df_t.drop_duplicates()
print(len(aut_df_t))

73260


In [34]:
aut_df.to_csv(f'../data/{version}/Book_Author.tsv', sep='\t', index=False)

In [80]:
aut_df_t.to_csv(f'../data/{version}/Author.tsv', sep='\t', index=False)

# Genre

In [36]:
categories = data[['ISBN', 'categories']]
cat_arr = []
for idx, row in categories.iterrows():
    try :
        cat_sp = ast.literal_eval(row['categories'])[0].split(', ')
        for a in cat_sp:
            cat_arr.append((row['ISBN'], a))
    except :
        continue

In [37]:
cat_df = pd.DataFrame(cat_arr, columns=['book_id','genre_id'])

In [38]:
enc = LabelEncoder()

In [39]:
cats = cat_df['genre_id']

In [40]:
cat_df['genre_id'] = enc.fit_transform(cat_df['genre_id'])

In [42]:
cat_df

,book_id,genre_id
0,0001713353,363
1,0001712799,746
2,0002005263,1231
3,0002005263,3031
4,0001381733,2182
...,...,...
123706,1947279424,822
123707,0615622526,1282
123708,0692699422,4459
123709,1548371270,3588


In [43]:
cat_df_t = pd.DataFrame()

In [44]:
cat_df_t['id'] = cat_df['genre_id']

In [45]:
cat_df_t['name'] = cats

In [46]:
cat_df_t = cat_df_t.drop_duplicates()

In [47]:
print(len(cat_df_t))

4474


In [48]:
cat_df_t

,id,name
0,363,Animals
1,746,Bedtime
2,1231,Chee
3,3031,Jim (Fictitious character)
4,2182,English poetry
...,...,...
123638,2639,Guerrilla warfare
123668,1959,Dissertations
123669,74,Academic
123704,3953,Riddles


In [49]:
cat_df.to_csv(f'../data/{version}/Book_Genre.tsv', sep='\t', index=False)

In [50]:
cat_df_t.to_csv(f'../data/{version}/Genre.tsv', sep='\t', index=False)

# Rating

In [51]:
def load_rating():
    gc.collect()
    with open(f'../data/books.json', 'r') as json_file :
        for line in json_file :
            data = json.loads(line)
            try : name = data['reviewerName']
            except : name = 'Jhonny Jhon'
            yield data['reviewerID'], data['asin'], data['overall'], name

In [52]:
rat_arr = []
i = 0
for data in tqdm(load_rating()):
    rat_arr.append(data)

51311621it [09:12, 92828.54it/s] 


In [53]:
rating = pd.DataFrame(rat_arr, columns=['user','item', 'rating', 'name'])

In [54]:
rating

,user,item,rating,name
0,A1C6M8LCIX4M6M,0001713353,5.0,June Bug
1,A1REUF3A1YCPHM,0001713353,5.0,TW Ervin II
2,A1YRBRK2XM5D5,0001713353,5.0,Rebecca L. Menner
3,A1V8ZR5P78P4ZU,0001713353,5.0,Mindy Stone
4,A2ZB06582NXCIV,0001713353,5.0,B. Deniger
...,...,...,...,...
51311616,A3P8PUZFHWFK1E,B01HJDS76Y,4.0,Kindle Customer
51311617,A26Q9T9G9MTX9S,B01HJDS76Y,1.0,Kindle Customer
51311618,AB9M1MQPBAS2J,B01HJEB422,5.0,Heather Hundley
51311619,A2TO93KMH3DJIK,B01HJEB422,5.0,WB


In [55]:
books.id

0         0001713353
1         0001712799
2         0002005263
3         0001381733
4         0001711296
             ...    
121559    1947279424
121560    0615622526
121561    0692699422
121562    1548371270
121563    0830845488
Name: id, Length: 121564, dtype: object

In [56]:
rating = rating.loc[rating['item'].isin(books.id)]

In [57]:
rating

,user,item,rating,name
0,A1C6M8LCIX4M6M,0001713353,5.0,June Bug
1,A1REUF3A1YCPHM,0001713353,5.0,TW Ervin II
2,A1YRBRK2XM5D5,0001713353,5.0,Rebecca L. Menner
3,A1V8ZR5P78P4ZU,0001713353,5.0,Mindy Stone
4,A2ZB06582NXCIV,0001713353,5.0,B. Deniger
...,...,...,...,...
51274634,A1EFPCDCPHMOGK,1720316910,5.0,fruity
51274635,A3KNX6OH9S8N5K,1720316910,5.0,Shelly
51274636,A1MK9OPMKYRMN7,1720316910,5.0,Amazon Customer
51274637,A2OLPD6EKPO6XB,1720316910,5.0,gooseneck


In [58]:
rating['user'] = rating['user'] + '@gmail.com'

/tmp/ipykernel_109598/597605893.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating['user'] = rating['user'] + '@gmail.com'


In [59]:
rating

,user,item,rating,name
0,A1C6M8LCIX4M6M@gmail.com,0001713353,5.0,June Bug
1,A1REUF3A1YCPHM@gmail.com,0001713353,5.0,TW Ervin II
2,A1YRBRK2XM5D5@gmail.com,0001713353,5.0,Rebecca L. Menner
3,A1V8ZR5P78P4ZU@gmail.com,0001713353,5.0,Mindy Stone
4,A2ZB06582NXCIV@gmail.com,0001713353,5.0,B. Deniger
...,...,...,...,...
51274634,A1EFPCDCPHMOGK@gmail.com,1720316910,5.0,fruity
51274635,A3KNX6OH9S8N5K@gmail.com,1720316910,5.0,Shelly
51274636,A1MK9OPMKYRMN7@gmail.com,1720316910,5.0,Amazon Customer
51274637,A2OLPD6EKPO6XB@gmail.com,1720316910,5.0,gooseneck


In [60]:
rating_df = rating[['item', 'user', 'rating']]

In [61]:
user_info = rating[['user', 'name']]

In [62]:
rating_df

,item,user,rating
0,0001713353,A1C6M8LCIX4M6M@gmail.com,5.0
1,0001713353,A1REUF3A1YCPHM@gmail.com,5.0
2,0001713353,A1YRBRK2XM5D5@gmail.com,5.0
3,0001713353,A1V8ZR5P78P4ZU@gmail.com,5.0
4,0001713353,A2ZB06582NXCIV@gmail.com,5.0
...,...,...,...
51274634,1720316910,A1EFPCDCPHMOGK@gmail.com,5.0
51274635,1720316910,A3KNX6OH9S8N5K@gmail.com,5.0
51274636,1720316910,A1MK9OPMKYRMN7@gmail.com,5.0
51274637,1720316910,A2OLPD6EKPO6XB@gmail.com,5.0


In [63]:
user_info['hashed_password'] = 1111

/tmp/ipykernel_109598/4114960031.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_info['hashed_password'] = 1111


In [64]:
user_info = user_info[['user', 'hashed_password', 'name']]

In [67]:
user_info.rename(columns= {'user' : 'id'}, inplace=True)

In [68]:
user_info = user_info.drop_duplicates()

In [74]:
user_info['idx'] = user_info['name'].str.find(' (')
for i in tqdm(user_info['name'].index) : 
    user_info.loc[i, 'name'] = user_info.loc[i, 'name'][:user_info.loc[i,'idx']]

/tmp/ipykernel_109598/1552534938.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_info['idx'] = user_info['name'].str.find(' (')
100%|██████████| 7386557/7386557 [14:46<00:00, 8329.32it/s]


In [75]:
user_info.drop('idx', axis=1, inplace=True)

/tmp/ipykernel_109598/40912406.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_info.drop('idx', axis=1, inplace=True)


In [69]:
rating_df = rating_df.drop_duplicates(subset=['user','item'])

In [76]:
user_info

,id,hashed_password,name
0,A1C6M8LCIX4M6M@gmail.com,1111,June Bu
1,A1REUF3A1YCPHM@gmail.com,1111,TW Ervin I
2,A1YRBRK2XM5D5@gmail.com,1111,Rebecca L. Menne
3,A1V8ZR5P78P4ZU@gmail.com,1111,Mindy Ston
4,A2ZB06582NXCIV@gmail.com,1111,B. Denige
...,...,...,...
51273751,AIVBMXYCVSFLN@gmail.com,1111,Adrion Kusan
51274633,A3H1FPWMDEDBOW@gmail.com,1111,Kindle CustomerJudyann Phillip
51274635,A3KNX6OH9S8N5K@gmail.com,1111,Shell
51274636,A1MK9OPMKYRMN7@gmail.com,1111,Amazon Custome


In [71]:
rating_df

,item,user,rating
0,0001713353,A1C6M8LCIX4M6M@gmail.com,5.0
1,0001713353,A1REUF3A1YCPHM@gmail.com,5.0
2,0001713353,A1YRBRK2XM5D5@gmail.com,5.0
3,0001713353,A1V8ZR5P78P4ZU@gmail.com,5.0
4,0001713353,A2ZB06582NXCIV@gmail.com,5.0
...,...,...,...
51274634,1720316910,A1EFPCDCPHMOGK@gmail.com,5.0
51274635,1720316910,A3KNX6OH9S8N5K@gmail.com,5.0
51274636,1720316910,A1MK9OPMKYRMN7@gmail.com,5.0
51274637,1720316910,A2OLPD6EKPO6XB@gmail.com,5.0


In [72]:
rating_df.to_csv(f'../data/{version}/Rating.tsv', sep='\t', index=False)

In [77]:
user_info.to_csv(f'../data/{version}/User_Info.tsv', sep='\t', index=False)

In [83]:
19526995 / (7386557 * 123711) * 100

0.0021369041578282464